In [1]:
from strategyTree import genStrategyTree
from valuations import *
from wordLists import guesses, answers

In [2]:
fullList = guesses + answers

In [3]:
tree = genStrategyTree(fullList, maxSizeSplit)

Depth: 0:   0%|          | 0/12947 [00:00<?, ?it/s]

Guess: serai, Remaining: 12947


  Depth: 1:   0%|          | 0/12947 [00:00<?, ?it/s]

  Guess: lawed, Remaining: 421


    Depth: 2:   0%|          | 0/12947 [00:00<?, ?it/s]

...

    Guess: swear, Remaining: 2
      Leaf: skear, depth = 3
      Leaf: swear, depth = 3
    Leaf: stear, depth = 2
    Leaf: shear, depth = 2
    Leaf: smear, depth = 2
    Leaf: spear, depth = 2


  Depth: 1:   0%|          | 0/12947 [00:00<?, ?it/s]

  Guess: flong, Remaining: 6
    Leaf: sofar, depth = 2
    Leaf: sowar, depth = 2
    Leaf: symar, depth = 2
    Leaf: solar, depth = 2
    Leaf: sonar, depth = 2
    Leaf: sugar, depth = 2
  Leaf: solei, depth = 1
  Leaf: tarsi, depth = 1
  Leaf: teras, depth = 1
  Leaf: uraei, depth = 1
  Leaf: uteri, depth = 1


  Depth: 1:   0%|          | 0/12947 [00:00<?, ?it/s]

  Guess: raise, Remaining: 2
    Leaf: arise, depth = 2
    Leaf: raise, depth = 2


In [4]:
import json

In [5]:
with open('maxSizeSplit.json', 'w') as f:
    json.dump(tree, f, sort_keys = True, indent=4)